In [1]:
# Install required libraries
!pip install transformers torchaudio librosa sentencepiece datasets

# Import libraries
import os
import librosa
import numpy as np
import torchaudio
from datasets import load_dataset, DatasetDict
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, pipeline
import torch


  Obtaining dependency information for torchaudio from https://files.pythonhosted.org/packages/6a/97/1780e3dd8733da30ff1051b8cbd8006e4824b76028558a58c31e790c09cd/torchaudio-2.5.1-cp311-cp311-macosx_11_0_arm64.whl.metadata


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 14.6 MB/s eta 0:00:00a 0:00:01


/Users/krishnakumarthankappan/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
/Users/krishnakumarthankappan/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(


In [3]:
# Path to the dataset
dataset_path = "/Users/krishnakumarthankappan/Documents/papers and chapters/KK/voice data project medium"

In [2]:
import os
print(os.getcwd())

/Users/krishnakumarthankappan/Documents/papers and chapters/KK/voice data project medium


In [5]:
pip install librosa torchaudio transformers pandas numpy datasets sentencepiece


Note: you may need to restart the kernel to use updated packages.


In [6]:
import os
import pandas as pd
import librosa
import numpy as np
import torchaudio
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC, pipeline
import torch


In [15]:
# Base path to your dataset
base_path = "/Users/krishnakumarthankappan/Documents/papers and chapters/KK/voice data project medium/cv-corpus-20.0-delta-2024-12-06"

# Path to metadata and audio clips
metadata_path = os.path.join(base_path, "untitled")  # File named "untitled"
audio_clips_path = os.path.join(base_path, "clips")


In [16]:
# Rename the file if needed
if os.path.isfile(metadata_path):
    os.rename(metadata_path, os.path.join(base_path, "train.tsv"))
metadata_path = os.path.join(base_path, "train.tsv")


In [18]:
import os

# Define the dataset path
dataset_path = "/Users/krishnakumarthankappan/Documents/papers and chapters/KK/voice data project medium/cv-corpus-20.0-delta-2024-12-06"

# List all files and directories
print("Contents of dataset path:")
print(os.listdir(dataset_path))


Contents of dataset path:
['.DS_Store', 'en']


In [19]:
en_path = "/Users/krishnakumarthankappan/Documents/papers and chapters/KK/voice data project medium/cv-corpus-20.0-delta-2024-12-06/en"

# List contents of the `en` directory
print("Contents of 'en' directory:")
print(os.listdir(en_path))

Contents of 'en' directory:
['validated_sentences.tsv', '.DS_Store', 'clips', 'unvalidated_sentences.tsv', 'validated.tsv', 'clip_durations.tsv', 'other.tsv', 'reported.tsv', 'invalidated.tsv']


In [20]:
import os

# Define paths
base_path = "/Users/krishnakumarthankappan/Documents/papers and chapters/KK/voice data project medium/cv-corpus-20.0-delta-2024-12-06/en"
metadata_path = os.path.join(base_path, "validated.tsv")
audio_clips_path = os.path.join(base_path, "clips")


In [21]:
import pandas as pd

# Load metadata
metadata = pd.read_csv(metadata_path, sep="\t")

# Display the first few rows
print("Metadata preview:")
print(metadata.head())


Metadata preview:
                                           client_id  \
0  031903093b6fa1aeb0a243843eb9ed57baf6e99d1f8f92...   
1  058fe5b1170aa09ef3f1092b179384639bc46ac53c1675...   
2  08190396a5c298331813531d1a832b56d8ffe44aaedcb7...   
3  14698ee63cabe08b43f0faa93304202d1e6ffeaa2cdf86...   
4  28d8f8a88afad9eb9e5b36ee84bd4c5ba84137310da15f...   

                           path  \
0  common_voice_en_41383256.mp3   
1  common_voice_en_41823983.mp3   
2  common_voice_en_41881685.mp3   
3  common_voice_en_41799514.mp3   
4  common_voice_en_41552032.mp3   

                                         sentence_id  \
0  f19a785911b1a3b1338e3eb5cc785e58b8381d21ec7c33...   
1  f50360e1be367d8155b3c8340f0b3d38d1e6701df79dc5...   
2  f4f3a5714cc36a9abbabf78a33feb4a9c368005f1f4bf5...   
3  f4d04f6e48777c3ad180c629858a19fdfa4cb875d2bb22...   
4  f262ed293fa5fe0986d1e7a80b5bbae11205f8089a1857...   

                                            sentence  sentence_domain  \
0  The outer rim has und

In [22]:
# Check audio files in the clips directory
audio_files = [f for f in os.listdir(audio_clips_path) if f.endswith(('.wav', '.mp3'))]
print("Sample audio files:", audio_files[:5])


Sample audio files: ['common_voice_en_41585622.mp3', 'common_voice_en_41517340.mp3', 'common_voice_en_41680084.mp3', 'common_voice_en_41774111.mp3', 'common_voice_en_41669154.mp3']


In [23]:
import librosa

# Preprocess a single audio file
def load_audio(row):
    audio_path = os.path.join(audio_clips_path, row["path"])
    speech, sr = librosa.load(audio_path, sr=16000)  # Resample to 16 kHz
    return speech, row["sentence"]

# Example: Load the first audio file
sample_audio, sample_text = load_audio(metadata.iloc[0])
print("Sample Audio Shape:", sample_audio.shape)
print("Sample Text:", sample_text)


Sample Audio Shape: (77184,)
Sample Text: The outer rim has undergone some erosion due to subsequent impacts.


In [24]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
import torch

# Load Wav2Vec2 model and processor
model_name = "facebook/wav2vec2-large-960h"
processor = Wav2Vec2Processor.from_pretrained(model_name)
model = Wav2Vec2ForCTC.from_pretrained(model_name)

# Transcribe audio
def transcribe_audio(audio):
    inputs = processor(audio, sampling_rate=16000, return_tensors="pt", padding=True)
    with torch.no_grad():
        logits = model(inputs.input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.decode(predicted_ids[0])
    return transcription

# Test transcription
sample_transcription = transcribe_audio(sample_audio)
print("Original Transcription:", sample_transcription)


/Users/krishnakumarthankappan/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

/Users/krishnakumarthankappan/anaconda3/lib/python3.11/site-packages/transformers/modeling_utils.py:479: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(chec

Original Transcription: THE OUTER RIM HAS UNDERGONE SOME EROSION DUE TO SUBSEQUENT IMPACTS


In [25]:
from transformers import pipeline

# Load text correction model
correction_model = pipeline("text2text-generation", model="vennify/t5-base-grammar-correction")

# Correct transcription
def correct_text(text):
    corrected = correction_model(text, max_length=50)
    return corrected[0]["generated_text"]

# Test correction
corrected_transcription = correct_text(sample_transcription)
print("Corrected Transcription:", corrected_transcription)


config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

/Users/krishnakumarthankappan/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(


pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

Corrected Transcription: The outer perimeter has undergone some erosion due to subsequent IMPACTS.


In [26]:
# Process the dataset
results = []
for _, row in metadata.iterrows():
    audio_path = os.path.join(audio_clips_path, row["path"])
    speech, _ = librosa.load(audio_path, sr=16000)
    transcription = transcribe_audio(speech)
    corrected = correct_text(transcription)
    results.append({"audio_path": audio_path, "transcription": transcription, "corrected": corrected})

# Save results to a DataFrame
results_df = pd.DataFrame(results)
print(results_df.head())

# Save results to a CSV file
results_df.to_csv("processed_transcriptions.csv", index=False)


                                          audio_path  \
0  /Users/krishnakumarthankappan/Documents/papers...   
1  /Users/krishnakumarthankappan/Documents/papers...   
2  /Users/krishnakumarthankappan/Documents/papers...   
3  /Users/krishnakumarthankappan/Documents/papers...   
4  /Users/krishnakumarthankappan/Documents/papers...   

                                       transcription  \
0  THE OUTER RIM HAS UNDERGONE SOME EROSION DUE T...   
1  FOR THE PURPOSES OF THIS DEFINITION THE INTENT...   
2  BINNOT WAS EDUCATED AT LANSWOOD HIGH SCHOOL LE...   
3  THESE LUUS BECAME NAHARS AUD MER LORD MOUNTIBE...   
4  THE GROUPING TRADITIONALLY CALLED APES IS BRAC...   

                                           corrected  
0  The outer perimeter has undergone some erosion...  
1  For the purposes of this definition, the INTEN...  
2  The BINNOT WAS EDUCATED AT LANSWOOD HIGH SCHOO...  
3  These LUUS BECAME NAHARS AUD MER LORD MOUNTIBE...  
4  The group of traditionally labelled APES IS BR..

In [27]:
from sklearn.model_selection import train_test_split

# Split the dataset into train, validation, and test sets
train_data, temp_data = train_test_split(metadata, test_size=0.3, random_state=42)
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

# Print dataset sizes
print(f"Training set size: {len(train_data)}")
print(f"Validation set size: {len(val_data)}")
print(f"Test set size: {len(test_data)}")


Training set size: 175
Validation set size: 37
Test set size: 38


In [28]:
# Function to preprocess audio
def load_audio_batch(data):
    audio_data = []
    for _, row in data.iterrows():
        audio_path = os.path.join(audio_clips_path, row["path"])
        speech, _ = librosa.load(audio_path, sr=16000)
        audio_data.append({"audio": speech, "text": row["sentence"]})
    return audio_data

# Load data
train_audio = load_audio_batch(train_data)
val_audio = load_audio_batch(val_data)
test_audio = load_audio_batch(test_data)



In [32]:
from jiwer import wer

# Function to calculate WER
def calculate_wer(true_texts, predicted_texts):
    return wer(true_texts, predicted_texts)


In [30]:
pip install jiwer


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  Obtaining dependency information for jiwer from https://files.pythonhosted.org/packages/dc/7f/00bec152973661ea89628490367fa3058d7f56d51d2c1ad02a44589d12cd/jiwer-3.0.5-py3-none-any.whl.metadata
  Obtaining dependency information for click<9.0.0,>=8.1.3 from https://files.pythonhosted.org/packages/7e/d4/7ebdbd03970677812aac39c869717059dbb71a4cfc033ca6e5221787892c/click-8.1.8-py3-none-any.whl.metadata
  Obtaining dependency information for rapidfuzz<4,>=3 from https://files.pythonhosted.org/packages/4c/55/d026c01c9312c9c2a413679052a9bb884743fc5655e59339116d83a2125b/rapidfuzz-3.11.0-cp311-cp311-macosx_11_0_arm64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9

In [31]:
from jiwer import wer, cer

# Test a simple WER calculation
reference = "this is a test"
hypothesis = "this is test"
print("WER:", wer(reference, hypothesis))


WER: 0.25


In [33]:
from jiwer import cer

# Function to calculate CER
def calculate_cer(true_texts, predicted_texts):
    return cer(true_texts, predicted_texts)


In [34]:
# Function to evaluate the model
def evaluate_model(audio_data):
    true_texts = []
    predicted_texts = []
    for data in audio_data:
        transcription = transcribe_audio(data["audio"])
        true_texts.append(data["text"])
        predicted_texts.append(transcription)
    
    # Calculate metrics
    wer_score = calculate_wer(true_texts, predicted_texts)
    cer_score = calculate_cer(true_texts, predicted_texts)
    
    return {"WER": wer_score, "CER": cer_score}

# Evaluate on validation set
val_metrics = evaluate_model(val_audio)
print(f"Validation WER: {val_metrics['WER']}")
print(f"Validation CER: {val_metrics['CER']}")

# Evaluate on test set
test_metrics = evaluate_model(test_audio)
print(f"Test WER: {test_metrics['WER']}")
print(f"Test CER: {test_metrics['CER']}")


Validation WER: 1.0191256830601092
Validation CER: 0.8229885057471265
Test WER: 1.0124688279301746
Test CER: 0.8273710482529119


In [35]:
# Save transcriptions and metrics
def save_results(audio_data, output_file):
    results = []
    for data in audio_data:
        transcription = transcribe_audio(data["audio"])
        corrected_transcription = correct_text(transcription)
        results.append({
            "true_text": data["text"],
            "transcription": transcription,
            "corrected_transcription": corrected_transcription
        })
    results_df = pd.DataFrame(results)
    results_df.to_csv(output_file, index=False)
    print(f"Results saved to {output_file}")

# Save validation and test results
save_results(val_audio, "validation_results.csv")
save_results(test_audio, "test_results.csv")


Results saved to validation_results.csv
Results saved to test_results.csv


In [39]:
!pip install git+https://github.com/openai/whisper.git

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  Cloning https://github.com/openai/whisper.git to /private/var/folders/8f/hsvvz4012sq0h_c9d9ph0g1m0000gn/T/pip-req-build-ufs6ru3s
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /private/var/folders/8f/hsvvz4012sq0h_c9d9ph0g1m0000gn/T/pip-req-build-ufs6ru3s
  Resolved https://github.com/openai/whisper.git to commit 90db0de1896c23cbfaf0c58bc2d30665f709f170
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Obtaining dependency information for tiktoken from https://files.pythonhosted.org/packages/8c/f8/f0101d98d661b34534769c3818f5af631e59c3

In [40]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.8/48.8 kB 3.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [41]:
import os
import whisper
import pandas as pd
from sklearn.model_selection import train_test_split


In [42]:
# Define paths
base_path = "/Users/krishnakumarthankappan/Documents/papers and chapters/KK/voice data project medium/cv-corpus-20.0-delta-2024-12-06/en"
metadata_path = os.path.join(base_path, "validated.tsv")
audio_clips_path = os.path.join(base_path, "clips")

# Load metadata
metadata = pd.read_csv(metadata_path, sep="\t")
print(metadata.head())

# Split the dataset
train_data, temp_data = train_test_split(metadata, test_size=0.3, random_state=42)
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)
print(f"Training set size: {len(train_data)}")
print(f"Validation set size: {len(val_data)}")
print(f"Test set size: {len(test_data)}")


                                           client_id  \
0  031903093b6fa1aeb0a243843eb9ed57baf6e99d1f8f92...   
1  058fe5b1170aa09ef3f1092b179384639bc46ac53c1675...   
2  08190396a5c298331813531d1a832b56d8ffe44aaedcb7...   
3  14698ee63cabe08b43f0faa93304202d1e6ffeaa2cdf86...   
4  28d8f8a88afad9eb9e5b36ee84bd4c5ba84137310da15f...   

                           path  \
0  common_voice_en_41383256.mp3   
1  common_voice_en_41823983.mp3   
2  common_voice_en_41881685.mp3   
3  common_voice_en_41799514.mp3   
4  common_voice_en_41552032.mp3   

                                         sentence_id  \
0  f19a785911b1a3b1338e3eb5cc785e58b8381d21ec7c33...   
1  f50360e1be367d8155b3c8340f0b3d38d1e6701df79dc5...   
2  f4f3a5714cc36a9abbabf78a33feb4a9c368005f1f4bf5...   
3  f4d04f6e48777c3ad180c629858a19fdfa4cb875d2bb22...   
4  f262ed293fa5fe0986d1e7a80b5bbae11205f8089a1857...   

                                            sentence  sentence_domain  \
0  The outer rim has undergone some erosio

In [43]:
# Load the Whisper model
model = whisper.load_model("base")  # You can replace "base" with "small", "medium", or "large"

100%|███████████████████████████████████████| 139M/139M [00:03<00:00, 41.2MiB/s]
/Users/krishnakumarthankappan/anaconda3/lib/python3.11/site-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any

In [44]:
# Function to transcribe audio
def transcribe_audio_whisper(audio_path):
    result = model.transcribe(audio_path)
    return result["text"]

# Example: Transcribe a single audio file
sample_audio_path = os.path.join(audio_clips_path, train_data.iloc[0]["path"])
transcription = transcribe_audio_whisper(sample_audio_path)
print("Transcription:", transcription)


/Users/krishnakumarthankappan/anaconda3/lib/python3.11/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Transcription:  The majority of the station's programming is produced and broadcast from Nottingham.


In [45]:
# Prepare audio dataset for evaluation
def prepare_audio_dataset(data):
    dataset = []
    for _, row in data.iterrows():
        audio_path = os.path.join(audio_clips_path, row["path"])
        dataset.append({"audio_path": audio_path, "text": row["sentence"]})
    return dataset

val_audio = prepare_audio_dataset(val_data)
test_audio = prepare_audio_dataset(test_data)


In [46]:
from jiwer import wer, cer

# Evaluate the model on WER and CER
def evaluate_whisper_model(audio_dataset):
    true_texts = []
    predicted_texts = []
    for data in audio_dataset:
        transcription = transcribe_audio_whisper(data["audio_path"])
        true_texts.append(data["text"])
        predicted_texts.append(transcription)
    
    wer_score = wer(true_texts, predicted_texts)
    cer_score = cer(true_texts, predicted_texts)
    return {"WER": wer_score, "CER": cer_score}

# Evaluate on validation and test sets
val_metrics = evaluate_whisper_model(val_audio)
test_metrics = evaluate_whisper_model(test_audio)

print(f"Validation WER: {val_metrics['WER']}")
print(f"Validation CER: {val_metrics['CER']}")
print(f"Test WER: {test_metrics['WER']}")
print(f"Test CER: {test_metrics['CER']}")


/Users/krishnakumarthankappan/anaconda3/lib/python3.11/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/Users/krishnakumarthankappan/anaconda3/lib/python3.11/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/Users/krishnakumarthankappan/anaconda3/lib/python3.11/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


...

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/Users/krishnakumarthankappan/anaconda3/lib/python3.11/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/Users/krishnakumarthankappan/anaconda3/lib/python3.11/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Validation WER: 0.23224043715846995
Validation CER: 0.09241379310344827
Test WER: 0.13466334164588528
Test CER: 0.045757071547420966


In [47]:
# Save transcription results
def save_results_whisper(audio_dataset, output_file):
    results = []
    for data in audio_dataset:
        transcription = transcribe_audio_whisper(data["audio_path"])
        results.append({"audio_path": data["audio_path"], "true_text": data["text"], "transcription": transcription})
    
    results_df = pd.DataFrame(results)
    results_df.to_csv(output_file, index=False)
    print(f"Results saved to {output_file}")

# Save validation and test results
save_results_whisper(val_audio, "whisper_validation_results.csv")
save_results_whisper(test_audio, "whisper_test_results.csv")


/Users/krishnakumarthankappan/anaconda3/lib/python3.11/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/Users/krishnakumarthankappan/anaconda3/lib/python3.11/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/Users/krishnakumarthankappan/anaconda3/lib/python3.11/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


...

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/Users/krishnakumarthankappan/anaconda3/lib/python3.11/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/Users/krishnakumarthankappan/anaconda3/lib/python3.11/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Results saved to whisper_test_results.csv
